In [2]:
import pandas as pd
xl = pd.ExcelFile("train.xlsx")
xl.sheet_names

df = xl.parse(0)
print(df.columns[2])

df = xl.parse(1)
print(df.columns)


print(len(df.a.dropna()))
df.c[0]

مشکل تغییر آوای انتظار
Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k'], dtype='object')
31


'سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون میشم. یه آوایی فعال کردم ولی نمیدونم که چیکار کنم که این نباشه یکی دیگه بجاش باشه هر کاری میکنم نمیشه'

In [3]:

alltext = ''
for i in df.columns:
    #print(len(df[i].dropna()))
    for j in df[i].dropna():
        alltext += j+' '

print(len(alltext))
print(len(alltext.split()))
print(len(set(alltext.split())))


alltext = alltext.replace("\u200c",'').replace("؟",' ? ').replace(".",' ').replace("،",' ')

rem = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
for i in rem:
    alltext = alltext.replace(i,' ')


40460
7978
1457


In [92]:
print(len(alltext))
print(len(alltext.split()))
print(len(set(alltext.split())))

#print(set(alltext.split()))
train_text = alltext.replace('  ',' ')

40738
8004
1332


In [5]:
d={'word':[],'counts':[]}

for i in (list(set(alltext.split()))):
    d['word']+=[i]
    d['counts']+= [alltext.count(i)]
counts = pd.DataFrame(d)
counts.sort_values(by=['counts'],ascending=False).head(10)

,word,counts
59,و,2232
834,ک,1045
794,س,726
485,ای,641
908,می,630
22,ام,554
259,ان,396
234,ع,373
854,از,344
605,کن,312


In [6]:
b = open('rawtext.txt','r',encoding='utf-8')
rawtext = b.read()


rem = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
for i in rem:
    rawtext = rawtext.replace(i,' ')
b = open('rawtext.txt','w',encoding='utf-8')
b.write(rawtext)
b.close()


In [7]:
print(len(rawtext))
print(len(rawtext.split()))
print(len(set(rawtext.split())))
print((rawtext.split()[10]))

3898814
699619
46604
۱۳۹۶


In [8]:
print(len(set(rawtext.split())|set(alltext.split()))-len(set(rawtext.split())))
(set(rawtext.split())|set(alltext.split()))-(set(rawtext.split()))
1

607


1

load raw tweet text data

In [82]:
raw_tweet = pd.read_csv("tweets.csv")
raw_tweet.columns[4]



C:\Users\HP\AppData\Local\Temp\ipykernel_576\885573527.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_tweet = pd.read_csv("tweets.csv")


'time'

In [32]:
raw_tweet.tweet[84209]
raw_tweet.tweet[2]


'کودک و نوجوان و جوان در کشور اسلامی می\u200cآموزد که استعداد و توان بالقوه\u200cی خود را برای ارزشهای متعالی ملی یعنی ارزشهای اسلامی و انقلابی شکوفا کند و به\u200cکار گیرد.\nاین آموزش حیاتی و سازوکار تحقق آن، همان کار بزرگ و جهاد مبارکی است که معلمان عهده\u200cدار آن شده\u200cاند./۴\n\n#روز_معلم'

In [59]:
raw_tweet_text = ''
for i in range (0,len(raw_tweet.tweet)-1,10):
    raw_tweet_text += raw_tweet.tweet[i]


In [275]:
rem = '✌️🔗😥😉0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\u2066،؟۹۶۲۱۰۳۷۵؛\u2069\xa0\u200c\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

rem = '\u2066،؟۹۶۲۱۰۳۷۵؛\u2069\xa0\u200c\'()*+,-./:;<=>?@_`'

list of train word for vectorizing input words

In [113]:
train_word_list = list(set(train_text.split()))
train_word_list.sort()
train_word_list[1325:1331]

['یکیش', 'یکیشو', 'یگانه', '۱۴۰۲', '۳۷۱۷۱', '۸۹']

In [276]:
out_data = []
#          [[0.5,0, -1331- ,0.1], -1331- ,[0.2, -1331- ,0.1]]

len_trainword = len(train_word_list)
for data_i in range (0,len_trainword):
    train_word = train_word_list[data_i]
    tweets_found_with_this_word = []
    out_data_line = []
    for i in range (0,len(raw_tweet.tweet),5):
        single_tweet = raw_tweet.tweet[i]
        

        if train_word in single_tweet:
            for j in rem:
                single_tweet = single_tweet.replace(j,' ')
            single_tweet = single_tweet.split(' ')
            tweets_found_with_this_word += single_tweet
            
    for data_i2 in range (0,len(train_word_list)):
        train_word = train_word_list[data_i2]
        out_data_line += [tweets_found_with_this_word.count(train_word)]
    out_data += [out_data_line]
    sumAll = sum(out_data[data_i])+1
    for data_i2 in range (0,len(train_word_list)):
        out_data[data_i][data_i2] = int((out_data[data_i][data_i2]/sumAll)*10000)/100
    out_data[data_i][data_i]=10


In [277]:
b = open('out_data_vector2.txt','w')
b.write(str(out_data))
b.close()

In [289]:
print( train_word )
#print(str(single_tweet))
print(train_word_list[94])   
#print(out_data[11][42])         
#print(out_data[11])
for i in range (1000,1330):
    print(train_word_list[i],out_data[94][i]) 

۸۹۸۹
امید
نشد 0.01
نشدم 0.01
نشدن 0.01
نشده 0.07
نشه 0.0
نظرتون 0.0
نفر 0.12
نفری 0.02
نفریم 0.01
نماز 0.0
نمازو 0.0
نمشه 0.0
نمود 0.01
نمونه 0.03
نمی 0.29
نمیاد 0.0
نمیتونم 0.0
نمیخوام 0.0
نمیخوامش 0.0
نمیخونه 0.0
نمیده 0.0
نمیدونستم 0.0
نمیدونم 0.01
نمیدونه 0.0
نمیزارن 0.0
نمیشه 0.0
نمیکنه 0.0
نمیکنید 0.0
نمیگیره 0.0
نه 0.18
نو 0.08
نوا 0.0
نواهنگ 0.0
نوای 0.01
نوتون 0.0
نوحه 0.0
نوروزتون 0.0
نوروزم 0.0
نوروزو 0.0
نوستالژیکه 0.0
نوم 0.0
نوی 0.0
نپرس 0.0
نکرد 0.0
نکردش 0.0
نکرده 0.02
نکن 0.01
نگاه 0.07
نیاد 0.0
نیس 0.0
نیست 0.24
نیستش 0.0
نیستم 0.02
نیومد 0.0
نیومده 0.0
ها 1.31
هاتون 0.0
هاتونو 0.0
های 2.15
هایی 0.16
هر 0.34
هرروشی 0.0
هرچی 0.0
هرکار 0.0
هرکاری 0.0
هرکی 0.0
هزینش 0.0
هزینه 0.03
هزینهاش 0.0
هست 0.14
هستش 0.0
هستند 0.23
هستو 0.0
هستید 0.01
هستیم 0.11
هستین 0.0
هفته 0.14
هم 1.14
همایون 0.0
همراه 0.11
همراهم 0.0
همراهو 0.0
همزمان 0.02
همسایه 0.0
همسرم 0.0
همسرمو 0.0
همش 0.0
همشو 0.0
هممون 0.0
همه 0.85
همون 0.01
همونجوری 0.0
همونه 0.0
همونو 0.0
همکارامون 0.0
همکارتون 0.0
ه